### Programers 연도별 대장균 크기 편차 구하기.

이 문제는 각 연도에서 가장 큰 대장균의 사이즈를 구하고,

그 값에서 변량들을 뺀 결과를 출력하는 문제.

### sample case 만 통과하고 test case all out.

```python
with MAX_SIZE_YEAR as (select max(SIZE_OF_COLONY) as MAX_SIZE,
                              DIFFERENTIATION_DATE
                       from ECOLI_DATA
                       group by DIFFERENTIATION_DATE)
                       
                       
select  extract(year from DIFFERENTIATION_DATE) as YEAR,
        (MAX_SIZE - SIZE_OF_COLONY) as YEAR_DEV,
        ID
        
from ECOLI_DATA

join MAX_SIZE_YEAR using (DIFFERENTIATION_DATE)

order by YEAR, YEAR_DEV asc;
```

원인을 분석해봤는데, 같은 연도가 많은 테이블일 경우 파티션 별 계산이 제대로 동작하지 않아서

테케가 나가는 거 같았는데, 문제는 cte 구문에서 

```python
max(SIZE_OF_COLONY) over(partition by DIFFERENTIATION_DATE)
```

를 할 경우 집계 함수를 썻더라도, group by clause 가 발생했는데 그 로직을 정확하게 모르겠다.

결국은 더 간단한 쿼리로 정답을 맞췄다.

### sample, test case all pass

```python
select extract(year from DIFFERENTIATION_DATE) as YEAR,
       (max(SIZE_OF_COLONY) over(partition by year(DIFFERENTIATION_DATE)) - SIZE_OF_COLONY) as YEAR_DEV,
       ID

from ECOLI_DATA

order by YEAR, YEAR_DEV asc;
```

결국은 위와 같이 파티션 별로 계산을 제대로 시킨 쿼리로 통과를 했는데 

위에 cte 구문 부분에서 group by clause 오류가 발생하는 이유를 찾아야 함...